In [6]:
import os
import Augmentor
import zipfile
import random
import zipfile
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile

In [2]:
local_zip = 'Discoloration.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('Dataset/')
zip_ref.close()

local_zip2 = 'Periodontal.zip'
zip_ref2    = zipfile.ZipFile(local_zip2, 'r')
zip_ref2.extractall('Dataset/') 
zip_ref2.close()

In [3]:
print(len(os.listdir('Dataset/Dental Discoloration')))
print(len(os.listdir('Dataset/Periodontal')))

440
417


In [4]:
def AugmentData(source_dir, count):
    source_dir = source_dir
    output_dir = "."
    p = Augmentor.Pipeline(source_directory=source_dir, output_directory=output_dir)
    #p.random_distortion(probability=1, grid_width=4, grid_height=4, magnitude=1)
    p.rotate(probability=0.001, max_left_rotation=0.001, max_right_rotation=0.001)
    #p.zoom_random(probability=0.5, percentage_area=0.9)
    #p.crop_random(probability=0.6, percentage_area=0.9)
    #p.resize(probability=1.0, width=64, height=64)
    p.sample(count)

In [5]:
AugmentData("Dataset/Discoloration/", 1000)
AugmentData("Dataset/Periodontal/", 1000)

OSError: The source directory you specified does not exist.

In [ ]:
AugmentData("Dataset/Healthy/", 1000)

In [ ]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[:testing_length]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)


Discoloration_SOURCE_DIR = "Dataset/Dental Discoloration/"
Periodontal_SOURCE_DIR = "Dataset/Periodontal/"
Healthy_SOURCE_DIR = "Dataset/Healthy/"

TRAINING_Discoloration_DIR = "Dataset_Real/training/Dental Discoloration/"
TRAINING_Periodontal_DIR = "Dataset_Real/training/Periodontal/"
TRAINING_Healthy_DIR = "Dataset_Real/training/Healthy/"

TESTING_Discoloration_DIR = "Dataset_Real/testing/Dental Discoloration/"
TESTING_Periodontal_DIR = "Dataset_Real/testing/Periodontal/"
TESTING_Healthy_DIR = "Dataset_Real/testing/Healthy/"

split_size = .8
split_data(Discoloration_SOURCE_DIR, TRAINING_Discoloration_DIR, TESTING_Discoloration_DIR, split_size)
split_data(Periodontal_SOURCE_DIR, TRAINING_Periodontal_DIR, TESTING_Periodontal_DIR, split_size)
split_data(Healthy_SOURCE_DIR, TRAINING_Healthy_DIR, TESTING_Healthy_DIR, split_size)

In [7]:
TRAINING_DIR = "Dataset_Real/training/"
# Experiment with your own parameters here to really try to drive it to 99.9% accuracy or better
train_datagen = ImageDataGenerator(rescale=1./255,
      rotation_range=50,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.1,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=10,
                                                    class_mode='categorical',
                                                    target_size=(150, 150))

VALIDATION_DIR = "Dataset_Real/testing/"
# Experiment with your own parameters here to really try to drive it to 99.9% accuracy or better
validation_datagen = ImageDataGenerator(rescale=1./255,
      rotation_range=50,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.1,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=8,
                                                              class_mode='categorical',
                                                              target_size=(150, 150))

# Expected Output:
# Found 22498 images belonging to 2 classes.
# Found 2500 images belonging to 2 classes.

Found 960 images belonging to 3 classes.
Found 240 images belonging to 3 classes.


In [19]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

# Print the model summary
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 74, 74, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_12 (Conv2D)          (None, 72, 72, 64)        36928     
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 36, 36, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 17, 17, 128)     

In [20]:
# Set the training parameters
model.compile(loss = 'categorical_crossentropy', 
              optimizer=Adam(learning_rate=0.0001), 
              metrics=['accuracy'])

In [21]:
history = model.fit(train_generator,
                    steps_per_epoch=32,
                    epochs=15,
                    validation_data=validation_generator,
                    validation_steps=10,
                    verbose=1)

Epoch 1/15
32/32 [==============================] - 2s 60ms/step - loss: 1.1148 - accuracy: 0.3781 - val_loss: 1.0503 - val_accuracy: 0.4500
Epoch 2/15
32/32 [==============================] - 2s 56ms/step - loss: 1.0217 - accuracy: 0.5250 - val_loss: 0.9624 - val_accuracy: 0.6125
Epoch 3/15
32/32 [==============================] - 2s 58ms/step - loss: 0.9005 - accuracy: 0.6031 - val_loss: 0.9533 - val_accuracy: 0.5000
Epoch 4/15
32/32 [==============================] - 2s 59ms/step - loss: 0.8452 - accuracy: 0.6125 - val_loss: 0.8677 - val_accuracy: 0.5750
Epoch 5/15
32/32 [==============================] - 2s 56ms/step - loss: 0.8804 - accuracy: 0.5938 - val_loss: 0.8156 - val_accuracy: 0.6250
Epoch 6/15
32/32 [==============================] - 2s 58ms/step - loss: 0.8080 - accuracy: 0.6156 - val_loss: 0.6567 - val_accuracy: 0.6500
Epoch 7/15
32/32 [==============================] - 2s 55ms/step - loss: 0.7390 - accuracy: 0.6812 - val_loss: 0.6821 - val_accuracy: 0.7125
Epoch 8/15
32